# Training of the Baseline Model

## Initialization steps

### load configuration

In [ ]:
# load configuration
import yaml
import os

with open(os.environ['BMPC_CONFIG'], 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)

### weights and biases

In [5]:
# initialize weights and biases

import wandb
# from wandb.keras import WandbCallback
from wandb.integration.keras import WandbCallback

project_name = 'baseline model'
wandb.init(project=project_name)

config = wandb.config

2024-05-30 17:49:19.055825: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 17:49:19.102079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 17:49:19.102126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 17:49:19.104164: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 17:49:19.112959: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 17:49:19.113824: I tensorflow/core/platform/cpu_feature_guard.cc:1

### hyperparameters

In [6]:
# hyperparameters

config.seq_length = 30
config.batch_size = 2
# config.val_ratio = 0.2
config.learning_rate = 1.0e-4
config.epochs = 1

### dataset

*Here, I currently import the "small" dataset that Joel had in his script.*

**TODO:** Find baseline dataset and load that instead.

In [7]:
# load dataset
from dlomix.data import FragmentIonIntensityDataset

# from misc import PTMS_ALPHABET
from dlomix.constants import PTMS_ALPHABET

# path to dataset
datset_base_path = "/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean"
dataset_train_path = f"{datset_base_path}_train.parquet"
dataset_val_path = f"{datset_base_path}_val.parquet"
dataset_test_path = f"{datset_base_path}_test.parquet"

dataset = FragmentIonIntensityDataset(
    data_source=dataset_train_path,
    val_data_source=dataset_val_path,
    test_data_source=dataset_test_path,
    data_format="parquet", 
    # val_ratio=config.val_ratio, # why do we need this if we already have splits?
    batch_size=config.batch_size,
    max_seq_len=config.seq_length,
    encoding_scheme="naive-mods",
    alphabet=PTMS_ALPHABET,
    # model_features=["precursor_charge", "collision_energy", "fragmentation_type"]
    model_features=["precursor_charge_onehot", "collision_energy_aligned_normed","method_nbr"]
)


/nfs/home/students/j.poschenrieder/MaPra_Wilhelm/dlomix2/src/dlomix/data/dataset.py:282: UserWarning: 
                Multiple data sources or a single non-train data source provided {'train': '/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean_train.parquet', 'val': '/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean_val.parquet', 'test': '/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean_test.parquet'}, please ensure that the data sources are already split into train, val and test sets
                since no splitting will happen. If not, please provide only one data_source and set the val_ratio to split the data into train and val sets."
                
  warnings.warn(
Casting the dataset: 100%|██████████| 2000/2000 [00:00<00:00, 54521.40 examples/s]


In [13]:
import tensorflow as tf
dataset.tensor_train_data.features
# tf.cast(dataset.tensor_train_data, tf.float32)
# for x in dataset.tensor_train_data:
#     print(x)
#     break

AttributeError: '_PrefetchDataset' object has no attribute 'features'

In [43]:
import tensorflow as tf
import numpy as np
min_val = np.Infinity
max_val = -np.Infinity
for x in dataset.tensor_val_data.as_numpy_iterator():
    seq = x[0]['modified_sequence']
    min_val = np.min([min_val, np.min(seq)])
    max_val = np.max([max_val, np.max(seq)])

print(f"min: {min_val}, max: {max_val}")

/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/datasets/utils/tf_utils.py:131: RuntimeWarning: invalid value encountered in cast
  array = array.astype(cast_dtype)


min: -9.223372036854776e+18, max: 56.0


### tensorflow setup

In [13]:
# initialize relevant stuff for training
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=20,
    restore_best_weights=True)

### model setup

In [19]:
# initialize model
from dlomix.models import PrositIntensityPredictor

input_mapping = {
        "SEQUENCE_KEY": "modified_sequence",
        "COLLISION_ENERGY_KEY": "collision_energy_aligned_normed",
        "PRECURSOR_CHARGE_KEY": "precursor_charge_onehot",
        "FRAGMENTATION_TYPE_KEY": "method_nbr",
    }

# meta_data_keys=["COLLISION_ENERGY_KEY","PRECURSOR_CHARGE_KEY","FRAGMENTATION_TYPE_KEY"]
meta_data_keys=["collision_energy_aligned_normed", "precursor_charge_onehot", "method_nbr"]

model = PrositIntensityPredictor(
    seq_length=config.seq_length,
    alphabet=PTMS_ALPHABET,
    use_prosit_ptm_features=False,
    with_termini=False,
    input_keys=input_mapping,
    meta_data_keys=meta_data_keys
)

model.compile(
    optimizer=optimizer,
    loss=masked_spectral_distance,
    metrics=[masked_pearson_correlation_distance]
)


NameError: name 'dataset' is not defined

## Training

In [20]:
model.fit(
    dataset.tensor_train_data,
    validation_data=dataset.tensor_val_data,
    epochs=config.epochs,
    callbacks=[WandbCallback(), early_stopping]
)

3996/3996 [==============================] - ETA: 0s - loss: 0.6062 - masked_pearson_correlation_distance: 0.4687

/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/datasets/utils/tf_utils.py:131: RuntimeWarning: invalid value encountered in cast
  array = array.astype(cast_dtype)


InvalidArgumentError: Graph execution error:

Detected at node prosit_intensity_predictor_5/embedding_5/embedding_lookup defined at (most recent call last):
  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/runpy.py", line 86, in _run_code

  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/traitlets/config/application.py", line 982, in launch_instance

  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start

  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start

  File "/nfs/home

--- Logging error ---
Traceback (most recent call last):
  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 122] Disk quota exceeded
Call stack:
  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 48, in run
    self._run()
  File "/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 99, in _run

In [21]:
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f3bb015a8f0>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f3bb015a8f0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe